In [ ]:
import pandas as pd
import tomllib

from data.data_processing import select_rename_columns

In [ ]:
CONFIG_PATH = './data/config.toml'
def load_config(section_name: str) -> dict:
    with open(CONFIG_PATH, 'rb') as config_file:
        return tomllib.load(config_file)[section_name]

In [ ]:
df = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')

def population_preprocessing(df: pd.DataFrame, table: str='statistiques_pop') -> pd.DataFrame:
    population_config = load_config(table)

    df = df.copy()[~df['CODGEO'].str.startswith('97')]

    rename_mapper = {key: value for key, value in population_config.items() if isinstance(value, str)}
    df = df[list(population_config.keys())].rename(
        columns=rename_mapper
    )
    
    temp_dfs = []
    statistics_mapper = {key: value for key, value in population_config.items() if isinstance(value, list)}
    nbr_max_params = sum(map(len, statistics_mapper.values()))
    for column, fields in statistics_mapper.items():
        temp_dfs.append(
            pd.DataFrame(
                {name: df[name] for name in rename_mapper.values()} | {
                    'annee_debut': fields[1],
                    'annee_fin': fields[1] if len(fields) < nbr_max_params else fields[2],
                    'type_statistique': fields[0],
                    'valeur': df[column].astype(float)
                }
            )
        )

    return pd.concat(temp_dfs, ignore_index=True)

population_preprocessing(df)

In [ ]:
df = pd.read_csv('./data/raw_data/area/v_commune_2023.csv', dtype='object')
df1 = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')


df1[~df1['CODGEO'].isin(df[df['TYPECOM'] == 'ARM']['COM'])]

('90001', 'population', '2024', '2024', 1),
('90002', 'population', '2024', '2024', 1),
('90003', 'population', '2024', '2024', 1),
('90004', 'population', '2024', '2024', 1),
('90005', 'population', '2024', '2024', 1),
('90006', 'population', '2024', '2024', 1),
('90007', 'population', '2024', '2024', 1),
('90008', 'population', '2024', '2024', 1),
('90009', 'population', '2024', '2024', 1),
('90010', 'population', '2024', '2024', 1),
('90011', 'population', '2024', '2024', 1),
('90012', 'population', '2024', '2024', 1),
('90013', 'population', '2024', '2024', 1),
('90014', 'population', '2024', '2024', 1),
('90015', 'population', '2024', '2024', 1),
('90016', 'population', '2024', '2024', 1),
('90017', 'population', '2024', '2024', 1),
('90018', 'population', '2024', '2024', 1),
('90019', 'population', '2024', '2024', 1),
('90020', 'population', '2024', '2024', 1),
('90021', 'population', '2024', '2024', 1),
('90022', 'population', '2024', '2024', 1),
('90023', 'population', '2024', '2024', 1),
('90024', 'population', '2024', '2024', 1),
('90025', 'population', '2024', '2024', 1),
('90026', 'population', '2024', '2024', 1),
('90027', 'population', '2024', '2024', 1),
('90028', 'population', '2024', '2024', 1),
('90029', 'population', '2024', '2024', 1),
('90030', 'population', '2024', '2024', 1),
('90031', 'population', '2024', '2024', 1),
('90032', 'population', '2024', '2024', 1),
('90033', 'population', '2024', '2024', 1),
('90034', 'population', '2024', '2024', 1),
('90035', 'population', '2024', '2024', 1),
('90036', 'population', '2024', '2024', 1),
('90037', 'population', '2024', '2024', 1),
('90039', 'population', '2024', '2024', 1),
('90041', 'population', '2024', '2024', 1),
('90042', 'population', '2024', '2024', 1),
('90043', 'population', '2024', '2024', 1),
('90044', 'population', '2024', '2024', 1),
('90045', 'population', '2024', '2024', 1),
('90046', 'population', '2024', '2024', 1),
('90047', 'population', '2024', '2024', 1),
('90048', 'population', '2024', '2024', 1),
('90049', 'population', '2024', '2024', 1),
('90050', 'population', '2024', '2024', 1),
('90051', 'population', '2024', '2024', 1),
('90052', 'population', '2024', '2024', 1),
('90053', 'population', '2024', '2024', 1),
('90054', 'population', '2024', '2024', 1),
('90055', 'population', '2024', '2024', 1),
('90056', 'population', '2024', '2024', 1),
('90057', 'population', '2024', '2024', 1),
('90058', 'population', '2024', '2024', 1),
('90059', 'population', '2024', '2024', 1),
('90060', 'population', '2024', '2024', 1),
('90061', 'population', '2024', '2024', 1),
('90062', 'population', '2024', '2024', 1),
('90063', 'population', '2024', '2024', 1),
('90064', 'population', '2024', '2024', 1),
('90065', 'population', '2024', '2024', 1),
('90066', 'population', '2024', '2024', 1),
('90067', 'population', '2024', '2024', 1),
('90068', 'population', '2024', '2024', 1),
('90069', 'population', '2024', '2024', 1),
('90070', 'population', '2024', '2024', 1),
('90071', 'population', '2024', '2024', 1),
('90072', 'population', '2024', '2024', 1),
('90074', 'population', '2024', '2024', 1),
('90075', 'population', '2024', '2024', 1),
('90076', 'population', '2024', '2024', 1),
('90077', 'population', '2024', '2024', 1),
('90078', 'population', '2024', '2024', 1),
('90079', 'population', '2024', '2024', 1),
('90080', 'population', '2024', '2024', 1),
('90081', 'population', '2024', '2024', 1),
('90082', 'population', '2024', '2024', 1),
('90083', 'population', '2024', '2024', 1),
('90084', 'population', '2024', '2024', 1),
('90085', 'population', '2024', '2024', 1),
('90086', 'population', '2024', '2024', 1),
('90087', 'population', '2024', '2024', 1),
('90088', 'population', '2024', '2024', 1),
('90089', 'population', '2024', '2024', 1),
('90090', 'population', '2024', '2024', 1),
('90091', 'population', '2024', '2024', 1),
('90093', 'population', '2024', '2024', 1),
('90094', 'population', '2024', '2024', 1),
('90095', 'population', '2024', '2024', 1),
('90096', 'population', '2024', '2024', 1),
('90097', 'population', '2024', '2024', 1),
('90098', 'population', '2024', '2024', 1),
('90099', 'population', '2024', '2024', 1),
('90100', 'population', '2024', '2024', 1),
('90101', 'population', '2024', '2024', 1),
('90102', 'population', '2024', '2024', 1),
('90103', 'population', '2024', '2024', 1),
('90104', 'population', '2024', '2024', 1),
('90105', 'population', '2024', '2024', 1);

In [ ]:
print(('(\'' + pd.DataFrame({'a': ['Je suis', 'une', 'banante']})['a'] + '\', \'population\', \'2024\', \'2024\', 1)').str.cat(sep=',\n') + ";")